In [2]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
#读取中文文件路径 
#f= open("F:/桌面文件/数据之美/data.xls")

data= pd.read_excel("F:/桌面文件/数据之美/data.xls")
data.head(5)#查看前五条

,id,读者证号,读者姓名,读者单位,借出状态,条形码,正题名,借出时刻,限还时刻,实还时刻,...,出版者,出版日期,获得方式,作品语种,单位,date_borrow,time_borrow,date_return,time_retrun,weeks
0,0,101217101,李,1012171/化学工程与工艺（专升本）/材料与化工学院,已还,207480769,东北往事,2019.03.10 12:54:55,2019.04.24 12:54:48,2019.04.03 13:22:23,...,重庆出版社,2009.05,28.0,chi,材料与化工学院,2019.03.10,12,2019.04.03,13.0,3.0
1,1,101217103,李,1012171/化学工程与工艺（专升本）/材料与化工学院,已还,205532137,化工厂的设计与改造,2019.03.30 16:35:13,2019.05.14 16:35:00,2019.03.30 16:35:53,...,中南大学出版社,2005.02,20.0,chi,材料与化工学院,2019.03.30,16,2019.03.30,16.0,0.0
2,2,101217103,李,1012171/化学工程与工艺（专升本）/材料与化工学院,未还,205667526,化工原理课程设计,2019.03.30 16:35:13,2019.05.14 16:35:26,NaN,...,大连理工大学出版社,2005.02,14.8,chi,材料与化工学院,2019.03.30,16,NaN,NaN,NaN
3,3,101217103,李,1012171/化学工程与工艺（专升本）/材料与化工学院,未还,203897426,化工计算,2019.03.30 16:35:14,2019.05.14 16:35:37,NaN,...,化学工业出版社,1990.06,24.0,chi,材料与化工学院,2019.03.30,16,NaN,NaN,NaN
4,4,101217107,李,1012171/化学工程与工艺（专升本）/材料与化工学院,已还,210620194,化工原理,2018.09.05 19:03:40,2018.12.04 19:03:40,2018.12.03 08:39:37,...,高等教育出版社,2010.06,29.0,chi,材料与化工学院,2018.09.05,19,2018.12.03,8.0,12.0


In [4]:
#查看字段信息
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31169 entries, 0 to 31168
Data columns (total 24 columns):
id             31169 non-null int64
读者证号           31169 non-null int64
读者姓名           31169 non-null object
读者单位           31169 non-null object
借出状态           31169 non-null object
条形码            31169 non-null int64
正题名            31167 non-null object
借出时刻           31169 non-null object
限还时刻           31169 non-null object
实还时刻           25619 non-null object
文献类型           31169 non-null object
分配地址           31169 non-null object
索取号            31167 non-null object
责任者            31011 non-null object
出版者            31109 non-null object
出版日期           30937 non-null object
获得方式           31141 non-null object
作品语种           30758 non-null object
单位             31169 non-null object
date_borrow    31169 non-null object
time_borrow    31169 non-null int64
date_return    25619 non-null object
time_retrun    25619 non-null float64
weeks          25619 non-null float64
dtype

In [5]:

#根据图书名分组
res2=data.groupby(['正题名'],as_index=False)['正题名'].agg({'借书数量':'count'})
res2.head(5)

,正题名,借书数量
0,1860.0,1
1,1984.0,4
2,1988.0,3
3,2018.0,4
4,"""5""是这样得来的",1


In [6]:
#排序（为后面的top20）
res2=res2.sort_values(by='借书数量',ascending=False).head(20)
res2

,正题名,借书数量
7833,明朝那些事儿,171
3161,化工原理课程设计,135
5914,平凡的世界,76
2551,傲慢与偏见,74
13896,高等数学,73
3235,化工设计,72
10052,生物化学,68
9013,河南省通用安装工程预算定额,66
9012,河南省房屋建筑与装饰工程预算定额,61
10385,盗墓笔记,59


In [7]:
data['分配地址'].head(3)

0    文学库108
1    自科库501
2    自科库501
Name: 分配地址, dtype: object

In [8]:
#取消相同文库的不同的地点
data['分配地址']=data['分配地址'].replace('[0-9]',' ',regex=True)
data['正题名']=data['正题名'].replace('[0-9]',' ',regex=True)
data['分配地址'].head(3)

0    文学库   
1    自科库   
2    自科库   
Name: 分配地址, dtype: object

In [9]:
#不同文库的借书分析
res1_manage=data.groupby(['分配地址'],as_index=False)['分配地址'].agg({'借书数量':'count'})
res1_manage=res1_manage.sort_values(by='借书数量',ascending=False).head(30)
res1_manage

,分配地址,借书数量
3,文学库,10177
9,自科库,9186
6,社科库,6157
10,语言库,2819
2,建筑库,2249
7,综合库,272
0,土木与交通工程学院资料室,181
1,建筑与城市规划学院资料室,124
4,理工二线,2
5,理工二线 （ A),1


In [10]:
#截取时间的年月
a=data['date_borrow'].str.partition('.').get(0)
al=data['date_borrow'].str.partition('.').get(2)
#data['date_borrow']
b=al.str.partition('.').get(0)
data['date_borrow']=a+"."+b
#print(data['date_borrow'])

In [11]:
data['date_borrow'].head(2)

0    2019.03
1    2019.03
Name: date_borrow, dtype: object

In [12]:
#data['date_borrow']根据年月分组
borrow=data.groupby(['date_borrow'],as_index=False)['date_borrow'].agg({'借书数量':'count'})
borrow

,date_borrow,借书数量
0,2018.04,3300
1,2018.05,2976
2,2018.06,2092
3,2018.07,1383
4,2018.08,460
5,2018.09,3365
6,2018.10,3030
7,2018.11,2944
8,2018.12,2831
9,2019.01,2408


In [13]:
res2=res2.sort_values(by='借书数量',ascending=True).head(20)
#借还书占比
btrue=data.groupby(['借出状态'],as_index=False)['借出状态'].agg({'是否还书':'count'})
a=btrue['是否还书'][0]/(btrue['是否还书'][0]+btrue['是否还书'][1])
a

0.821938464499984

In [14]:
#整体每日借还图
a=data['借出时刻'].str.partition(' ').get(2)
data["borrow_hour"]=a.str.partition(':').get(0)
b=data['实还时刻'].str.partition(' ').get(2)
data["return_hour"]=b.str.partition(':').get(0)

In [15]:
borrow_true_sum=data.groupby(['borrow_hour'],as_index=False)['borrow_hour'].agg({'borrow_true_sum':'count'})
return_hour_sum=data.groupby(['return_hour'],as_index=False)['borrow_hour'].agg({'return_hour_sum':'count'})
return_hour_sum

,return_hour,return_hour_sum
0,06,3
1,07,211
2,08,1134
3,09,2074
4,10,2000
5,11,2475
6,12,1462
7,13,1735
8,14,2232
9,15,3037


In [16]:
#统计有多少人借书
apr=data.groupby(['读者证号'],as_index=False)['正题名'].agg({'借书数量':'count'})
apr.head(3)

,读者证号,借书数量
0,20010201,14
1,20010302,12
2,20010310,6


In [17]:
#根据读者证号排序  为下面的相同ID的数据合并
da=data.sort_values(by='读者证号',ascending=True)
da.head(2)

,id,读者证号,读者姓名,读者单位,借出状态,条形码,正题名,借出时刻,限还时刻,实还时刻,...,获得方式,作品语种,单位,date_borrow,time_borrow,date_return,time_retrun,weeks,borrow_hour,return_hour
25768,25768,20010201,王,党委宣传部,未还,213471624,计算机网络简明教程,2018.06.25 10:02:26,2018.11.22 10:02:31,NaN,...,35.9,chi,党委宣传部,2018.06,10,NaN,NaN,NaN,10,NaN
25766,25766,20010201,王,党委宣传部,已还,210737443,英语笔译常用词语应试手册,2018.06.25 10:02:24,2018.11.22 10:02:24,2018.06.25 10:02:25,...,32.0,chi,党委宣传部,2018.06,10,2018.06.25,10.0,0.0,10,10


In [18]:
#重索引
da=da.reset_index(drop=True)
#删除Id
da=da.drop('id',axis=1)
da.head(3)

,读者证号,读者姓名,读者单位,借出状态,条形码,正题名,借出时刻,限还时刻,实还时刻,文献类型,...,获得方式,作品语种,单位,date_borrow,time_borrow,date_return,time_retrun,weeks,borrow_hour,return_hour
0,20010201,王,党委宣传部,未还,213471624,计算机网络简明教程,2018.06.25 10:02:26,2018.11.22 10:02:31,NaN,中文图书,...,35.9,chi,党委宣传部,2018.06,10,NaN,NaN,NaN,10,NaN
1,20010201,王,党委宣传部,已还,210737443,英语笔译常用词语应试手册,2018.06.25 10:02:24,2018.11.22 10:02:24,2018.06.25 10:02:25,中文图书,...,32.0,chi,党委宣传部,2018.06,10,2018.06.25,10.0,0.0,10,10
2,20010201,王,党委宣传部,未还,206473421,英语笔译综合能力(三级),2018.06.25 10:04:23,2018.11.22 10:04:54,NaN,中文图书,...,48.0,chi,党委宣传部,2018.06,10,NaN,NaN,NaN,10,NaN


In [19]:
#提取读者证号 和  图书名
da=pd.concat([da['读者证号'],da['正题名']],axis =1)
#data1 = np.array(da.groupby(['读者证号']))
da.head(3)

,读者证号,正题名
0,20010201,计算机网络简明教程
1,20010201,英语笔译常用词语应试手册
2,20010201,英语笔译综合能力(三级)


In [20]:
data1 = np.array(da.groupby(['读者证号']))
data1[2]

array([20010310,
               读者证号       正题名
26  20010310     红墙回忆上
27  20010310     红墙回忆下
28  20010310     我的前半生
29  20010310      两代悲歌
30  20010310  末代皇帝的后半生
31  20010310     李宗仁大传], dtype=object)

In [21]:
#设置id长度
id_kinds =len(apr)
all_value = []
id_kinds

3330

In [22]:
for j in range(id_kinds):
    value = []
    for i in data1[j][1]['正题名']:
        value.append(i)
    all_value.append(value)
print(all_value)

[['计算机网络简明教程', '英语笔译常用词语应试手册', '英语笔译综合能力(三级)', '英语笔译综合能力(三级)', '英语笔译综合能力(三级)', '英语口译全真试题精解', '英语笔译常用词语应试手册', '计算机网络', '英语笔译常用词语应试手册', '英语口译全真试题精解', '英语笔译常用词语应试手册', '如鱼得水记单词', '英语口译全真试题精解', '英语口译全真试题精解'], ['花', '自己的文章', '自己的文章', '自己的文章', '自己的文章', '故都的秋', '    年中篇小说', '呼啸山庄', '巴黎的鳞爪', '边城', '围城', '自己的文章'], ['红墙回忆上', '红墙回忆下', '我的前半生', '两代悲歌', '末代皇帝的后半生', '李宗仁大传'], ['红星照耀中国', '昆虫记', '红星照耀中国', '昆虫记', '红星照耀中国', '昆虫记', '红星照耀中国', '昆虫记', '昆虫记', '本草纲目', '天工开物', '哈利波特与火焰杯', '哈利波特与火焰杯', '哈利波特与凤凰社', '哈利波特与火焰杯', '哈利波特与凤凰社', '武媚娘秘史', '哈利波特与火焰杯', '哈利波特与凤凰社', '红星照耀中国'], ['中级无机化学', '配位化合物的立体化学', '无机化学核心教程', '无机化学', '动物庄园', '城堡', '人和吞食者', '带上她的眼睛', '普通无机化学', '     . ', '化学辞典', '锰分族、铁系、铂系', '氮磷砷分族第四卷', '希有气体、氢、碱金属', '锕系、锕系后无素', '《无机化学》第六卷卤素铜分族锌分族', '钛分族·钒分族·铬分族 第八卷', '氧硫硒硒分族', '无机化学丛书:铍硼铝镓分族第二卷', '碳硅锗分族', '基础有机化学上下册', '基础有机化学上下册', '配位化学', '  无机结构化学', '基础有机化学 ( 第 版 ) 习题解析', '高等无机化学', '基础有机化学', '物理化学学习指导', '无机化学', '基础有机化学', '     . ', '动物庄园', '带上她的眼睛', '人和吞食者', '城堡', '中国古建筑知识手册', '大壮·适形', '《营造法式》解读', '中国古建筑

In [23]:
len(all_value)

3330

In [24]:
new_data = pd.DataFrame({'id':range(len(all_value)),'value':all_value})
new_data

,id,value
0,0,"[计算机网络简明教程, 英语笔译常用词语应试手册, 英语笔译综合能力(三级), 英语笔译综合..."
1,1,"[花, 自己的文章, 自己的文章, 自己的文章, 自己的文章, 故都的秋, 年中篇小..."
2,2,"[红墙回忆上, 红墙回忆下, 我的前半生, 两代悲歌, 末代皇帝的后半生, 李宗仁大传]"
3,3,"[红星照耀中国, 昆虫记, 红星照耀中国, 昆虫记, 红星照耀中国, 昆虫记, 红星照耀中国..."
4,4,"[中级无机化学, 配位化合物的立体化学, 无机化学核心教程, 无机化学, 动物庄园, 城堡,..."
5,5,"[文苑故事, 盗墓笔记, 白云苍狗, 梦想与疯狂, 代理市长, 盗墓笔记, 扑通扑通,喜欢你..."
6,6,"[明朝那些事儿, 明朝那些事儿, 明朝那些事儿, 明朝那些事儿, 明朝那些事儿, 明朝那些事..."
7,7,"[西行漫记, 昆虫记, 红岩, 西行漫记]"
8,8,"[商店室内空间设计, 商店室内空间设计, 高速公路服务区规划--理论与实践, 家在香港 ,..."
9,9,"[Linux入门与实战经典, Linux C程序设计王者归来, Linux从入门到精通, 移..."


In [25]:
from numpy import * 
# 构造数据
def loadDataSet():    
    return  new_data['value'].values.tolist()[2:4]#[["书","笔","纸"], ["书","笔","手机"], ["书","笔","纸","手机","电脑"], ["手机","电脑"],["手机","电脑"],["手机","电脑"],["手机","电脑"]] 
# 将所有元素转换为frozenset型字典，存放到列表中
def createC1(dataSet):    
    C1 = []    
    for transaction in dataSet:        
        for item in transaction:            
            if not [item] in C1:                
                C1.append([item])    
    C1.sort()# 使用frozenset是为了后面可以将这些值作为字典的键    
    return list(map(frozenset, C1)) 
# frozenset一种不可变的集合，set可变集合 
# 过滤掉不符合支持度的集合# 返回 频繁项集列表retList 所有元素的支持度字典
def scanD(D, Ck, minSupport):    
    ssCnt = {}  
    for tid in D:        
        for can in Ck:            
            if can.issubset(tid):   
                # 判断can是否是tid的《子集》 （这里使用子集的方式来判断两者的关系）                
                if can not in ssCnt:    
                    # 统计该值在整个记录中满足子集的次数（以字典的形式记录，frozenset为键）                    
                    ssCnt[can] = 1                
                else:                    
                    ssCnt[can] += 1    
    numItems = float(len(D))    
    retList = []        
    # 重新记录满足条件的数据值（即支持度大于阈值的数据） 
    supportData = {}    
    # 每个数据值的支持度    
    for key in ssCnt:        
        support = ssCnt[key] / numItems        
        if support >= minSupport:            
            retList.insert(0, key)        
        supportData[key] = support    
    return retList, supportData 
# 排除不符合支持度元素后的元素 每个元素支持度 
# 生成所有可以组合的集合# 频繁项集列表Lk 项集元素个数k  [frozenset({2, 3}), frozenset({3, 5})] -> [frozenset({2, 3, 5})]
def aprioriGen(Lk, k):    
    retList = []    
    lenLk = len(Lk)    
    for i in range(lenLk): 
        # 两层循环比较Lk中的每个元素与其它元素        
        for j in range(i+1, lenLk):            
            L1 = list(Lk[i])[:k-2]  # 将集合转为list后取值            
            L2 = list(Lk[j])[:k-2]            
            L1.sort(); L2.sort()        
            # 这里说明一下：该函数每次比较两个list的前k-2个元素，如果相同则求并集得到k个元素的集合            
            if L1==L2:                
                retList.append(Lk[i] | Lk[j]) # 求并集    
    return retList  # 返回频繁项集列表Ck # 封装所有步骤的函数# 返回 所有满足大于阈值的组合 集合支持度列表
def apriori(dataSet, minSupport = 0.5):    
    D = list(map(set, dataSet)) # 转换列表记录为字典  [{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]    
    C1 = createC1(dataSet)      # 将每个元素转会为frozenset字典    [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]    
    L1, supportData = scanD(D, C1, minSupport)  # 过滤数据    
    L = [L1]    
    k = 2    
    while (len(L[k-2]) > 0):    # 若仍有满足支持度的集合则继续做关联分析        
        Ck = aprioriGen(L[k-2], k)  # Ck候选频繁项集        
        Lk, supK = scanD(D, Ck, minSupport) # Lk频繁项集        
        supportData.update(supK)    # 更新字典（把新出现的集合:支持度加入到supportData中）        
        L.append(Lk)        
        k += 1  # 每次新组合的元素都只增加了一个，所以k也+1（k表示元素个数）    
        return L, supportData 
    dataSet =loadDataSet()
    L,suppData = apriori(dataSet)
    print(L)
    print(suppData)

In [26]:
loadDataSet()[:1]
#pd.loadDataSet()

[['红墙回忆上', '红墙回忆下', '我的前半生', '两代悲歌', '末代皇帝的后半生', '李宗仁大传']]

In [27]:
new_data['value']=new_data['value'].replace('[0-9]','',regex=True)

In [30]:
dataSet =loadDataSet()#new_data['value'][0]=str(new_data['value'][0])
#apriori(dataSet)
dataSet[0:2]
#for i in range(len(da)-1):
#    if type(dataSet[i])!=str:
#        dataSet[i]=str(dataSet[i])
#        print(dataSet[i])

[['红墙回忆上', '红墙回忆下', '我的前半生', '两代悲歌', '末代皇帝的后半生', '李宗仁大传'],
 ['红星照耀中国',
  '昆虫记',
  '红星照耀中国',
  '昆虫记',
  '红星照耀中国',
  '昆虫记',
  '红星照耀中国',
  '昆虫记',
  '昆虫记',
  '本草纲目',
  '天工开物',
  '哈利波特与火焰杯',
  '哈利波特与火焰杯',
  '哈利波特与凤凰社',
  '哈利波特与火焰杯',
  '哈利波特与凤凰社',
  '武媚娘秘史',
  '哈利波特与火焰杯',
  '哈利波特与凤凰社',
  '红星照耀中国']]

In [31]:
apriori(dataSet)

([[frozenset({'红星照耀中国'}),
   frozenset({'武媚娘秘史'}),
   frozenset({'本草纲目'}),
   frozenset({'昆虫记'}),
   frozenset({'天工开物'}),
   frozenset({'哈利波特与火焰杯'}),
   frozenset({'哈利波特与凤凰社'}),
   frozenset({'红墙回忆下'}),
   frozenset({'红墙回忆上'}),
   frozenset({'李宗仁大传'}),
   frozenset({'末代皇帝的后半生'}),
   frozenset({'我的前半生'}),
   frozenset({'两代悲歌'})],
  [frozenset({'哈利波特与凤凰社', '哈利波特与火焰杯'}),
   frozenset({'哈利波特与凤凰社', '天工开物'}),
   frozenset({'哈利波特与火焰杯', '天工开物'}),
   frozenset({'哈利波特与凤凰社', '昆虫记'}),
   frozenset({'哈利波特与火焰杯', '昆虫记'}),
   frozenset({'天工开物', '昆虫记'}),
   frozenset({'哈利波特与凤凰社', '本草纲目'}),
   frozenset({'哈利波特与火焰杯', '本草纲目'}),
   frozenset({'天工开物', '本草纲目'}),
   frozenset({'昆虫记', '本草纲目'}),
   frozenset({'哈利波特与凤凰社', '武媚娘秘史'}),
   frozenset({'哈利波特与火焰杯', '武媚娘秘史'}),
   frozenset({'天工开物', '武媚娘秘史'}),
   frozenset({'昆虫记', '武媚娘秘史'}),
   frozenset({'本草纲目', '武媚娘秘史'}),
   frozenset({'哈利波特与凤凰社', '红星照耀中国'}),
   frozenset({'哈利波特与火焰杯', '红星照耀中国'}),
   frozenset({'天工开物', '红星照耀中国'}),
   frozenset({'昆虫记', '红星照耀中国'}),
   fro

In [32]:
# 获取关联规则的封装函数
def generateRules(L, supportData, minConf=0.7):  # supportData 是一个字典    
    bigRuleList = []    
    for i in range(1, len(L)):  # 从为2个元素的集合开始        
        for freqSet in L[i]:            # 只包含单个元素的集合列表            
            H1 = [frozenset([item]) for item in freqSet]    # frozenset({2, 3}) 转换为 [frozenset({2}), frozenset({3})]            # 如果集合元素大于2个，则需要处理才能获得规则            
            if (i > 1):                
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf) # 集合元素 集合拆分后的列表 。。。            
            else:                
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList # 对规则进行评估 获得满足最小可信度的关联规
def calcConf(freqSet, H, supportData, brl, minConf=0.7):    
    prunedH = []  # 创建一个新的列表去返回    
    for conseq in H:        
        conf = supportData[freqSet]/supportData[freqSet-conseq]  # 计算置信度        
        if conf >= minConf:            
            print(freqSet-conseq,'-->',conseq,'conf:',conf)            
            brl.append((freqSet-conseq, conseq, conf))            
            prunedH.append(conseq)    
    return prunedH # 生成候选规则集合
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):    
    m = len(H[0])    
    if (len(freqSet) > (m + 1)): # 尝试进一步合并        
        Hmp1 = aprioriGen(H, m+1) # 将单个集合元素两两合并        
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)        
        if (len(Hmp1) > 1):    #need at least two sets to merge            
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf) 
dataSet = loadDataSet()
L,suppData = apriori(dataSet,minSupport=0.5)
rules = generateRules(L,suppData,minConf=0.7)# rules = generateRules(L,suppData,minConf=0.5)print(rules)


frozenset({'哈利波特与凤凰社'}) --> frozenset({'哈利波特与火焰杯'}) conf: 1.0
frozenset({'哈利波特与火焰杯'}) --> frozenset({'哈利波特与凤凰社'}) conf: 1.0
frozenset({'哈利波特与凤凰社'}) --> frozenset({'天工开物'}) conf: 1.0
frozenset({'天工开物'}) --> frozenset({'哈利波特与凤凰社'}) conf: 1.0
frozenset({'天工开物'}) --> frozenset({'哈利波特与火焰杯'}) conf: 1.0
frozenset({'哈利波特与火焰杯'}) --> frozenset({'天工开物'}) conf: 1.0
frozenset({'哈利波特与凤凰社'}) --> frozenset({'昆虫记'}) conf: 1.0
frozenset({'昆虫记'}) --> frozenset({'哈利波特与凤凰社'}) conf: 1.0
frozenset({'哈利波特与火焰杯'}) --> frozenset({'昆虫记'}) conf: 1.0
frozenset({'昆虫记'}) --> frozenset({'哈利波特与火焰杯'}) conf: 1.0
frozenset({'天工开物'}) --> frozenset({'昆虫记'}) conf: 1.0
frozenset({'昆虫记'}) --> frozenset({'天工开物'}) conf: 1.0
frozenset({'哈利波特与凤凰社'}) --> frozenset({'本草纲目'}) conf: 1.0
frozenset({'本草纲目'}) --> frozenset({'哈利波特与凤凰社'}) conf: 1.0
frozenset({'哈利波特与火焰杯'}) --> frozenset({'本草纲目'}) conf: 1.0
frozenset({'本草纲目'}) --> frozenset({'哈利波特与火焰杯'}) conf: 1.0
frozenset({'天工开物'}) --> frozenset({'本草纲目'}) conf: 1.0
frozenset({'本草纲目'}) --> 

In [33]:
data["borrow_hour"].head(2)

0    12
1    16
Name: borrow_hour, dtype: object

BI可视化大屏展示


In [36]:
import os as os
from bs4 import BeautifulSoup
from pyecharts import Pie,Page,Bar,EffectScatter,Line,Overlap,Liquid
page=Page()
label_color=["#5AB5FF","#5AB5FF","#5AB5FF","#fffff","#eeeeee","#5AB5FF'","#000","#fffff","#eeeeee","#dddddd","#fffff","#eeeeee","#ddddd"]
pie = Pie("不同文库借书数量分布图",title_pos='center', width=900,title_text_size=22,title_color="#5AB5FF")#pie = Pie("TX", title_pos='left', width=900,title_text_size=40)
#pie.use_theme("dark")
pie.add("册数", res1_manage['分配地址'],res1_manage['借书数量'],is_label_show=True, radius=[20, 68], rosetype="area", center=[50, 55],label_text_size=12,is_legend_text_size=5,is_legend_show=False,is_toolbox_show=False,is_more_utils =False) 
# 定义玫瑰图的类型，可选有radius add()函数，通用配置项，不知道大家记住没有    “商品B”大家自行忽略......后面的is_legend_show将图例设为不显示了    name为不同的属性名称，在这里为：oil、gas、coal、clean energy    
#cols为从表格读出来的，不同属性对用的数据    center为调整饼图圆心坐标    is_random为是否随即排列颜色列表（bool）
#radius为半径，第一个为内半径，第二个是外半径    rosetype为是否展示成南丁格尔图        'radius' 圆心角展现数据半分比，半径展现数据大小        'area' 圆心角相同，为通过半径展现数据大小    is_label_show为是否显示标签（各个属性的数据信息）    label_text_size为调整标签字体大小
#pie.render('pie.html')
page.add_chart(pie,name="pie2")
page.add_chart(pie,name="pie")
#水平柱形图
lovers_rank_bar = Bar('TOP20借阅图书排行榜',width=1000,height=400,title_pos='center',title_color="#5AB5FF")
#lovers_rank_bar.use_theme('dark')
lovers_rank_bar.add('' ,res2['正题名'],res2['借书数量'],is_convert=True,is_label_show=True,label_pos='right',is_toolbox_show=False,yaxis_line_color="blue",xaxis_line_color='blue',label_color=label_color)#is_convert 是否水平
page.add_chart(lovers_rank_bar,name="lovers_rank_bar")


liquid =Liquid("借还书占比",title_pos='center',title_text_size='22',title_color="#5AB5FF")
liquid.add("借还书占比", [a,0.6,0.2,0.5], is_liquid_outline_show=True)
page.add_chart(liquid,name="liquid")

#散点图
es=EffectScatter("总体每日借还书分析",title_color="#5AB5FF",title_pos='center',title_text_size=22)
#s.add("arrow_sample",return_hour_sum['return_hour'],return_hour_sum['return_hour_sum'],symbol_size=12, effect_scale=4.5,label_text_color ='#5AB5FF',effect_period=4,symbol="pin")
es.add("借书",borrow_true_sum['borrow_hour'],borrow_true_sum['borrow_true_sum'],symbol_size=12, is_datazoom_show=True,effect_scale=1, effect_period=4,Gridlines=False,label_text_color ='#5AB5FF',is_legend_show=False,yaxis_line_color="blue",xaxis_line_color='blue',xaxis_rotate=30, yaxis_rotate=30)
es.add("还书",return_hour_sum['return_hour'],return_hour_sum['return_hour_sum'],symbol_size=12,label_color=label_color, is_datazoom_show=True,effect_scale=1.5, effect_period=4,symbol="pin",Gridlines=False,label_text_color ='#5AB5FF',is_legend_show=False,yaxis_line_color="blue",xaxis_line_color='blue',xaxis_rotate=30, yaxis_rotate=30)
#es.render()
line = Line()
line.add("还书" ,return_hour_sum['return_hour'],return_hour_sum['return_hour_sum'])
line.add("借书" ,borrow_true_sum['borrow_hour'],borrow_true_sum['borrow_true_sum'])
overlap2 = Overlap()
overlap2.add(es)
overlap2.add(line)
page.add_chart(overlap2,name="overlap2")
#
label_color=["#5AB5FF","#5AB5FF","#5AB5FF","#fffff","#eeeeee","#5AB5FF'","#000","#fffff","#eeeeee","#dddddd","#fffff","#eeeeee","#ddddd"]
bar = Bar("2018年4月--2019年4月借书情况",title_color="#5AB5FF",title_pos='center',title_text_size=22)
bar.add("数量" ,borrow['date_borrow'], borrow['借书数量'],mark_point=["max", "min"],is_label_show=True,is_label_emphasis=True,enderer='canvas',is_toolbox_show=False,
        is_more_utils = True,label_color=label_color,label_text_color ='#5AB5FF',label_pos='outside',is_legend_show=False,is_datazoom_show=True,yaxis_line_color="blue",xaxis_line_color='blue',xaxis_rotate=30, yaxis_rotate=30)
line = Line()
line.add("数量" ,borrow['date_borrow'], borrow['借书数量'])
overlap = Overlap()
overlap.add(bar)
overlap.add(line)
page.add_chart(overlap,name="overlap")


page.render("bi.html")
#读写HTML
with open(os.path.join(os.path.abspath("."),"bi.html"),'r+',encoding="utf8") as html:
    html_bf=BeautifulSoup(html,"lxml")
    divs=html_bf.find_all("div")
    divs[0]["style"]="width:25%;height:44%;position:absolute;top:8%;left:0%;border-style:solid;border-color:#444444;border-width:3px;"    #修改图表大小、位置、边框
    divs[1]["style"]="width:48%;height:55%;position:absolute;top:8%;left:26%;border-style:solid;border-color:#444444;border-width:3px;"  #修改图表大小、位置、边框
    divs[2]["style"]="width:25%;height:44%;position:absolute;top:8%;left:75%;border-style:solid;border-color:#444444;border-width:3px;"  #修改图表大小、位置、边框
    divs[3]["style"]="width:25%;height:44%;position:absolute;top:54%;left:0%;border-style:solid;border-color:#444444;border-width:3px;"  #修改图表大小、位置、边框
    divs[4]["style"]="width:25%;height:44%;position:absolute;top:54%;left:75%;border-style:solid;border-color:#444444;border-width:3px;"  #修改图表大小、位置、边框
    divs[5]["style"]="width:48%;height:33%;position:absolute;top:65%;left:26%;border-style:solid;border-color:#444444;border-width:3px;"  #修改图表大小、位置、边框
    body=html_bf.find("body")
    body["style"]="background-color:#333333;"
    div_title="<div  align=\"light\"  style=\"width:1800px;position:absolute;left:40%;\">\n<span style=\"font-size:32px;font face=\'黑体\';color:#5AB5FF\"><b>基于pyecharts的BI监控大屏</b></div>"    #修改页面背景色、追加标题
    body.insert(0,BeautifulSoup(div_title,"lxml").div)
    html_new=str(html_bf)
    html.seek(0,0)
    html.truncate()
    html.write(html_new)
    html.close()